In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# Specify the folder containing weather data files
data_folder = "weather_data"

# Create the "Temperature_plot" folder
os.makedirs("aggregated_weather_data", exist_ok=True)

# Initialize a dictionary to store data for each city
city_data = {}

In [14]:
# Iterate through each file in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".csv"):
        filepath = os.path.join(data_folder, filename)
        filename_without_extension = os.path.splitext(os.path.basename(filepath))[0]
        city_name, year = filename_without_extension.split("_")[0], filename_without_extension.split("_")[1]  # Extract city name and year

        # Read the CSV file into a DataFrame object
        df = pd.read_csv(filepath, encoding="cp1252")

        # Drop some columns by name
        df = df.drop(columns=['Temp Flag', 'Dew Point Temp Flag', 'Rel Hum Flag', 'Precip. Amount (mm)', 'Precip. Amount Flag',
                              'Wind Dir Flag', 'Wind Spd Flag', 'Visibility Flag', 'Stn Press Flag', 'Hmdx', 'Hmdx Flag', 'Wind Chill Flag'])
        df["Date"] = pd.to_datetime(df[["Year", "Month", "Day"]])

        # If the city is not in the dictionary, initialize a new DataFrame for it
        if city_name not in city_data:
            city_data[city_name] = df
        else:
            # Concatenate the new data with the existing DataFrame for the city
            city_data[city_name] = pd.concat([city_data[city_name], df], ignore_index=True)

In [15]:
# Save the combined data for each city into a single CSV and create a plot
for city, data in city_data.items():
    # Group by date and get the maximum and minimum temperature
    df_max = data.groupby("Date")["Temp (°C)"].max()
    df_min = data.groupby("Date")["Temp (°C)"].min()

    # Save to CSV
    combined_csv_path = f"aggregated_weather_data/{city}_aggregated_weather.csv"
    data.to_csv(combined_csv_path, index=False)
    
    # Create a plot for the city
    plt.figure(figsize=(10, 6))
    plt.plot(df_max.index, df_max, color='r', label='Maximum Temperature')
    plt.plot(df_min.index, df_min, color='b', label='Minimum Temperature')
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.title(f"Maximum and Minimum Temperature Over Time in {city}")
    plt.grid()
    plt.legend()
    plt.savefig(f"aggregated_weather_data/{city}_temperature_plot.png")  # Save the plot as an image
    plt.close()  # Close the current plot to create a new one

print("Combined CSVs saved and plots created successfully!")

Combined CSVs saved and plots created successfully!
